# 1. 환경 설정 및 패키지 설치

In [1]:
 # Install Pytorch & other libraries
 %pip install -q tensorboard wandb
 # Install Hugging Face libraries
 %pip install -q --upgrade \
    "transformers==4.45.1" \
    "datasets==3.0.1" \
    "accelerate==0.34.2" \
    "evaluate==0.4.3" \
    "bitsandbytes==0.44.0" \
    "trl==0.11.1" \
    "peft==0.13.0" \
    "qwen-vl-utils"
 %pip install "Pillow>=9.4.0"
 %pip install scikit-learn


[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import io
import json
import torch
from PIL import Image # 이미지 처리 위한 라이브러리
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import AutoModelForVision2Seq, AutoProcessor # 이미지와 텍스트를 함께 처리하는 모델, 특정 모델에 맞는 전처리기를 자동으로 불러옴
from trl import SFTConfig, SFTTrainer
from transformers import Qwen2VLProcessor # Qwen2-VL 모델 전용 전처리기
from qwen_vl_utils import process_vision_info # 비전 정보를 처리
from peft import LoraConfig
import wandb

wandb.init(mode= 'disabled')

# 2. 시스템 프롬프트 및 사용자 프롬프트 정의

In [3]:
system_message = "당신은 이미지와 제품명(name)으로부터 패션/ 스타일 정보를 추론하는 분류 모델입니다."

#실제로 사용자 입력 -> 모델이 답해야 하는 프롬프트
prompt = """ 입력 정보:
- name : {name}
- image : [image]

위 정보를 바탕으로, 아래 7가지 key에 대한 값을 JSON 형태로 추론해 주세요 :
1) gender
2) masterCategory
3) subCategory
4) season
5) usage
6) baseColour
7) articleType

출력 시 ** 아래 JSON 예시 형태**를 반드시 지키세요:
{{
    "gender" : "예시 값",
    "masterCategory" : "예시 값",
    "subCategory" : "예시 값",
    "season" : "예시 값",
    "usage" : "예시 값",
    "baseColour" : "예시 값",
    "articleType" : "예시 값",
}}

#예시
{{
    "gender" : "Men",
    "masterCategory" : "Accessories",
    "subCategory" : "Eyewear",
    "season" : "Winter",
    "usage" : "Causal",
    "baseColour" : "Blue",
    "articleType" : "Sunglasses",
}}
#주의
- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.
"""

# 3. 데이터 전처리 함수 정의

In [4]:
def combine_cols_to_label(example):  # 여러 칼럼에 분산된 패션 속성 정보를 하나의 JSON 형태 레이블로 통합
    # 실제 컬럼명에 맞게 수정
    label_dict = {
        "gender" : example["gender"],
        "masterCategory" : example["masterCategory"],
        "subCategory" : example["subCategory"],
        "season" : example["season"],
        "usage" : example["usage"],
        "baseColour" : example["baseColour"],
        "articleType" : example["articleType"],
    }
    example["label"] = json.dumps(label_dict, ensure_ascii = False) # json.dumps() -> dict, list 등을 JSON 문자열로 변환
    return example

def format_data(sample):  #OpenAI 형식의 대화 구조로 변환
    #Image.Image를 PngImageFile로 변환
    buffer = io.BytesIO()       # 메모리 상에서 이미지를 PNG 형태로 변환한 후 다시 로드
    sample["image"].save(buffer, format = "PNG")
    buffer.seek(0)
    image = Image.open(buffer)

    return {
        "messages" : [
            {"role" : "system",
             "content" : [
                 {
                     "type" : "text",
                     "text" : system_message
                 }
             ],
            },
            {"role" : "user",
             "content" : [
                 {
                     "type" : "text",
                     "text" : prompt.format(name = sample["productDisplayName"]),
                 },
                 {
                     "type" : "image",
                     "image" : image,
                 }
            ],
            },
            {
                "rple" : "assistant",
                "content" : [
                    {
                        "type" : "text",
                        "text" : sample["label"],
                    }
                ]
            }
        ]
    }

# 4. 데이터 셋 로드 및 전처리

In [5]:
dataset = load_dataset("ashraq/fashion-product-images-small", split = "train")
dataset_add_label = dataset.map(combine_cols_to_label)
dataset_add_label = dataset_add_label.shuffle(seed = 4242)

In [6]:
dataset_add_label[0]

{'id': 15516,
 'gender': 'Men',
 'masterCategory': 'Footwear',
 'subCategory': 'Flip Flops',
 'articleType': 'Flip Flops',
 'baseColour': 'Navy Blue',
 'season': 'Fall',
 'year': 2011.0,
 'usage': 'Casual',
 'productDisplayName': 'Rockport Men Altrezlp Navy Blue Flip Flops',
 'image': <PIL.Image.Image image mode=RGB size=60x80>,
 'label': '{"gender": "Men", "masterCategory": "Footwear", "subCategory": "Flip Flops", "season": "Fall", "usage": "Casual", "baseColour": "Navy Blue", "articleType": "Flip Flops"}'}

In [7]:
# OpenAI 형식으로 변환
formatted_dataset = [format_data(row) for row in dataset_add_label]

In [8]:
formatted_dataset[0]

{'messages': [{'role': 'system',
   'content': [{'type': 'text',
     'text': '당신은 이미지와 제품명(name)으로부터 패션/ 스타일 정보를 추론하는 분류 모델입니다.'}]},
  {'role': 'user',
   'content': [{'type': 'text',
     'text': ' 입력 정보:\n- name : Rockport Men Altrezlp Navy Blue Flip Flops\n- image : [image]\n\n위 정보를 바탕으로, 아래 7가지 key에 대한 값을 JSON 형태로 추론해 주세요 :\n1) gender\n2) masterCategory\n3) subCategory\n4) season\n5) usage\n6) baseColour\n7) articleType\n\n출력 시 ** 아래 JSON 예시 형태**를 반드시 지키세요:\n{\n    "gender" : "예시 값",\n    "masterCategory" : "예시 값",\n    "subCategory" : "예시 값",\n    "season" : "예시 값",\n    "usage" : "예시 값",\n    "baseColour" : "예시 값",\n    "articleType" : "예시 값",\n}\n\n#예시\n{\n    "gender" : "Men",\n    "masterCategory" : "Accessories",\n    "subCategory" : "Eyewear",\n    "season" : "Winter",\n    "usage" : "Causal",\n    "baseColour" : "Blue",\n    "articleType" : "Sunglasses",\n}\n#주의\n- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.\n'},
    {'type': 'image',
     'image': <PIL.PngImagePlugin.PngImageFi

# 5. 학습/테스트 데이터 분할

In [9]:
train_dataset, test_dataset = train_test_split(formatted_dataset, test_size = 0.9, random_state = 42)

In [10]:
print("학습 데이터의 개수:", len(train_dataset))
print("테스트 데이터의 개수:", len(test_dataset))

학습 데이터의 개수: 4407
테스트 데이터의 개수: 39665


# 6. 모델 및 프로세서 로드

In [11]:
# 허깅페이스 모델 ID
model_id = "Qwen/Qwen2-VL-7B-Instruct"

#모델과 프로세소 로드
model=  AutoModelForVision2Seq.from_pretrained(
    model_id,
    device_map = "auto",
    torch_dtype = torch.bfloat16,
)
processor = AutoProcessor.from_pretrained(model_id)

config.json: 0.00B [00:00, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

VLM 에서는 **토크나이저** 대신 **프로세서** 사용

# 7. 챗 템플릿 적용 확인

In [12]:
text = processor.apply_chat_template(
    train_dataset[0]["messages"], tokenizer = False, add_generation_prompt = False
)
print(text)

<|im_start|>system
당신은 이미지와 제품명(name)으로부터 패션/ 스타일 정보를 추론하는 분류 모델입니다.<|im_end|>
<|im_start|>user
 입력 정보:
- name : Mr.Men Men's Charcoal White T-shirt
- image : [image]

위 정보를 바탕으로, 아래 7가지 key에 대한 값을 JSON 형태로 추론해 주세요 :
1) gender
2) masterCategory
3) subCategory
4) season
5) usage
6) baseColour
7) articleType

출력 시 ** 아래 JSON 예시 형태**를 반드시 지키세요:
{
    "gender" : "예시 값",
    "masterCategory" : "예시 값",
    "subCategory" : "예시 값",
    "season" : "예시 값",
    "usage" : "예시 값",
    "baseColour" : "예시 값",
    "articleType" : "예시 값",
}

#예시
{
    "gender" : "Men",
    "masterCategory" : "Accessories",
    "subCategory" : "Eyewear",
    "season" : "Winter",
    "usage" : "Causal",
    "baseColour" : "Blue",
    "articleType" : "Sunglasses",
}
#주의
- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.
<|vision_start|><|image_pad|><|vision_end|><|im_end|>
<|im_start|>
{"gender": "Men", "masterCategory": "Apparel", "subCategory": "Topwear", "season": "Fall", "usage": "Casual", "baseColour": "Grey", "articleType": "T

# 8. LoRA 설정

In [13]:
peft_config = LoraConfig(
    # 모델 가중치에 LoRA 업데이트를 적용하는 정도를 조절하는 스케일링 계수
    lora_alpha = 128,
    # 과적합을 방지하기 위한 드롭아웃 비율 설정
    lora_dropout = 0.05,
    #LoRA의 순위(rank) - 저차원 행렬의 차원을 결정
    r = 256,
    # 편향 업데이트 여부 - 'none'은 편향을 업데이트 하지 않음
    bias = "none",
    # LoRA를 적용할 대상 모듈들 - 트랜스포머 모델의 주요 투영 레이어들
    target_modules = [
        "q_proj", # Query 투영 레이어
        "k_proj", # Key 투영 레이어
        "v_proj", # Value 투영 레이어
        "o_proj", # Output 투영 레이어
        "gate_proj", # FFN 게이트 투영 레이어
        "up_proj",  # FFN 상향 투영 레이어
        "down_proj", # FFN 하향 투영 레이어

    ],
    # 작업 유형 지정 - 인과적 언어 모델링(다음 토큰 예측)
    task_type = "CAUSAL_LM"
)

# 9. 학습 설정

In [14]:
args = SFTConfig(
    output_dir = "output_dir",
    num_train_epochs = 2,
    per_device_train_batch_size = 16,
    gradient_accumulation_steps = 8,
    gradient_checkpointing = True,
    optim = "adamw_torch_fused",        #최적화기
    logging_steps = 10,
    save_strategy = "steps",
    save_steps = 50,
    bf16 = True,
    learning_rate = 1e-4,
    max_grad_norm = 0.3,                #그레디언트 클리핑
    warmup_ratio = 0.03,
    lr_scheduler_type = "constant",     # 고정 학습률
    push_to_hub = False,
    remove_unused_columns = False,
    dataset_kwargs = {"skip_prepare_dataset" : True},
    report_to = None
)

# 10. 데이터 콜레이터 함수 정의

In [15]:
def collate_fn(examples):
    """
    텍스트와 이미지가 포함된 대화 데이터를 모델 학습에 적합한 형태로 변환하는 함수

    Args:
        examples : 각각 "messages" 키를 가진 딕셔너리들의 리스트
                messages는 role(system/user/assistant)과 content를 포함하는 대화 형태

    Returns:
        batch : 모델 학습에 사용할 수 있는 토큰화된 텍스트, 이미지, 라벨이 포함된 배치
    """

    # 1단계 : 텍스트 전처리 - 채팅 템플릿 적용
    # 각 예제의 messages를 모델 고유의 채팅 형식으롭 변환
    # 모델마다 다른 특수 토큰과 형식을 사용 (예 : <|im_start|>, <|system|> 등)
    texts = [processor.apply_chat_template(example["messages"], tokenize = False) for example in examples]

    # 2단계 : 이미지 데이터 추출 및 전처리
    # messages에서 이미지 정보를 추출하여 모델이 처리할 수 있는 형태로 변환
    # process_vision_info()는 messages에서 이미지를 찾아 적절한 형태로 변환해주는 함수
    image_inputs = [process_vision_info(example["messages"])[0] for example in examples]

    # 3단계 : 텍스트 토크나이징 + 이미지 인코딩
    # 텍스트를 토큰으로 변환하고 이미지를 인코딩하여 하나의 배치로 묶음
    # return_tensors = "pt" : PyTorch 텐서 형태로 변환
    # padding = True :  배치 내 모든 시퀀스를 같은 길이로 맞춤
    batch = processor(text = texts, images = image_inputs, padding = True, return_tensors = "pt")

    # 4단계 : 라벨 생성(손실 계산용)
    # input_ids를 복사하여 라벨로 사용(다음 토큰 예측 학습을 위함)
    labels = batch["input_ids"].clone()

    # 5단계 : 패딩 토큰 손실 계산에서 제외
    # 패딩된 부분은 실제 데이터가 아니므로 손실 계산에서 제외
    # -100으로 설정하면 CrossEntropyLoss에서 자동으로 무시됨
    labels[labels == processor.tokenizer.pad_token_id] = -100

    # 6단계 : 이미지 토큰 손실 계산에서 제외
    # 이미지 토큰은 예측 대상이 아니므로 손실 계산에서 제외
    if isinstance(processor, Qwen2VLProcessor):
        # Qwen2VL 모델에서 사용하는 특수 이미지 토큰들의 ID
        # 151652 : 이미지 시작 토큰, 151633: 이미지 종료 토큰, 151655: 이미지 패치 토큰
        image_tokens = [151652,151633,151655]
    else:
        # 다른 비전 - 언어 모델의 이미지 토큰 ID 추출
        image_tokens = [processor.tokenizer.convert_tokens_to_ids(processor.image_token)]

    # 이미지 토큰들을 손실 계산에서 제외(-100으로 설정)
    for image_token_id in image_tokens:
        labels[labels == image_token_id] = -100

    # 7단계 : 최종 배치에 라벨 추가
    # 모델 학습 시 손실 계산에 사용될 라벨을 배치에 추가
    batch["labels"] = labels

    return batch

In [16]:
# 단일 예시 확인
example = train_dataset[0]
print("단일 예시 데이터:")
print(example)

#collate_fn 테스트(배치 크기 1로)
batch = collate_fn([example])
print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("이미지 픽셀 형태:", batch["pixel_values"].shape)
print("레이블 형태:", batch["labels"].shape)

단일 예시 데이터:
{'messages': [{'role': 'system', 'content': [{'type': 'text', 'text': '당신은 이미지와 제품명(name)으로부터 패션/ 스타일 정보를 추론하는 분류 모델입니다.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': ' 입력 정보:\n- name : Mr.Men Men\'s Charcoal White T-shirt\n- image : [image]\n\n위 정보를 바탕으로, 아래 7가지 key에 대한 값을 JSON 형태로 추론해 주세요 :\n1) gender\n2) masterCategory\n3) subCategory\n4) season\n5) usage\n6) baseColour\n7) articleType\n\n출력 시 ** 아래 JSON 예시 형태**를 반드시 지키세요:\n{\n    "gender" : "예시 값",\n    "masterCategory" : "예시 값",\n    "subCategory" : "예시 값",\n    "season" : "예시 값",\n    "usage" : "예시 값",\n    "baseColour" : "예시 값",\n    "articleType" : "예시 값",\n}\n\n#예시\n{\n    "gender" : "Men",\n    "masterCategory" : "Accessories",\n    "subCategory" : "Eyewear",\n    "season" : "Winter",\n    "usage" : "Causal",\n    "baseColour" : "Blue",\n    "articleType" : "Sunglasses",\n}\n#주의\n- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.\n'}, {'type': 'image', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size

In [17]:
print("입력에 대한 정수 인코딩 결과:")
print(batch["input_ids"][0])

입력에 대한 정수 인코딩 결과:
tensor([151644,   8948,    198,  64795,  82528,  33704,  90667,  21329,  80573,
        138017,  79632,   3153,      8,  42039, 126558,  45104,    101,  92031,
            14,  79207,  44680,    225,    222,  32077,  60039,  18411,  57835,
        126605,  42905, 128618,  97929,  54070, 142713,  78952,     13, 151645,
           198, 151644,    872,    198,  42349,  60039,    510,     12,    829,
           549,   4392,   1321,    268,  11012,    594,   4864,  40465,   5807,
           350,  33668,    198,     12,   2168,    549,    508,   1805,   2533,
         80901,  60039,  18411,  81718, 144059,  42039,     11, 136646,    220,
            22,  19969,  21329,   1376,  19391, 128605,  93668,   4718, 141966,
         17380,  57835, 126605,  33883,  55673,  50302,   6260,     16,      8,
          9825,    198,     17,      8,   7341,   6746,    198,     18,      8,
          1186,   6746,    198,     19,      8,   3200,    198,     20,      8,
         10431,    198

In [18]:
print("레이블에 대한 정수 인코딩 결과:")
print(batch["labels"][0])

레이블에 대한 정수 인코딩 결과:
tensor([151644,   8948,    198,  64795,  82528,  33704,  90667,  21329,  80573,
        138017,  79632,   3153,      8,  42039, 126558,  45104,    101,  92031,
            14,  79207,  44680,    225,    222,  32077,  60039,  18411,  57835,
        126605,  42905, 128618,  97929,  54070, 142713,  78952,     13, 151645,
           198, 151644,    872,    198,  42349,  60039,    510,     12,    829,
           549,   4392,   1321,    268,  11012,    594,   4864,  40465,   5807,
           350,  33668,    198,     12,   2168,    549,    508,   1805,   2533,
         80901,  60039,  18411,  81718, 144059,  42039,     11, 136646,    220,
            22,  19969,  21329,   1376,  19391, 128605,  93668,   4718, 141966,
         17380,  57835, 126605,  33883,  55673,  50302,   6260,     16,      8,
          9825,    198,     17,      8,   7341,   6746,    198,     18,      8,
          1186,   6746,    198,     19,      8,   3200,    198,     20,      8,
         10431,    19

In [19]:
# 토큰 디코딩 예시 (입력 텍스트가 어떻게 변환되었는지 확인)
decoded_text = processor.tokenizer.decode(batch["input_ids"][0])
print("디코딩된 텍스트:")
print(decoded_text)

디코딩된 텍스트:
<|im_start|>system
당신은 이미지와 제품명(name)으로부터 패션/ 스타일 정보를 추론하는 분류 모델입니다.<|im_end|>
<|im_start|>user
 입력 정보:
- name : Mr.Men Men's Charcoal White T-shirt
- image : [image]

위 정보를 바탕으로, 아래 7가지 key에 대한 값을 JSON 형태로 추론해 주세요 :
1) gender
2) masterCategory
3) subCategory
4) season
5) usage
6) baseColour
7) articleType

출력 시 ** 아래 JSON 예시 형태**를 반드시 지키세요:
{
    "gender" : "예시 값",
    "masterCategory" : "예시 값",
    "subCategory" : "예시 값",
    "season" : "예시 값",
    "usage" : "예시 값",
    "baseColour" : "예시 값",
    "articleType" : "예시 값",
}

#예시
{
    "gender" : "Men",
    "masterCategory" : "Accessories",
    "subCategory" : "Eyewear",
    "season" : "Winter",
    "usage" : "Causal",
    "baseColour" : "Blue",
    "articleType" : "Sunglasses",
}
#주의
- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.
<|vision_start|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|vision_end|><|im_end|>
<|im_start|>
{"gender": "Men", "masterCategory": "Apparel", "subCategory": "Topwear", "

# 11. 학습 시작

In [20]:
trainer = SFTTrainer(
    model = model,
    args = args,
    train_dataset = train_dataset,
    data_collator = collate_fn,
    peft_config = peft_config,
    tokenizer = processor.tokenizer
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [21]:
trainer.train()
trainer.save_model()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
10,0.580600
20,0.095100
30,0.079500
40,0.073500
50,0.068500
60,0.066100


Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
